---
# Check if GPU is available

In [1]:
import torch
torch.cuda.is_available()

True

---
# Logger

In [2]:
from lightning.pytorch.loggers import NeptuneLogger
from dotenv import load_dotenv
import os

API_TOKEN = os.environ.get("LOGGER_API")
PROJECT_NAME = 'mrobay/Audio-project'

logger_config = {
    "api_key": API_TOKEN,
    "project_name": PROJECT_NAME,
    "log_model_checkpoints": False
}

# Train

In [ ]:
import importlib
importlib.reload(importlib.import_module("hebrewASR"))
importlib.reload(importlib.import_module("dataModule"))
from hebrewASR import HebrewASR, train_func, default_config
neptune_logger = NeptuneLogger(project=PROJECT_NAME, api_key=API_TOKEN, log_model_checkpoints=False, tags=["train", "100-deepspeech"])
trainer = train_func(config=default_config, logger=neptune_logger, num_epochs=500)

# Resume Training

In [3]:
import importlib
importlib.reload(importlib.import_module("hebrewASR"))
importlib.reload(importlib.import_module("dataModule"))
from hebrewASR import HebrewASR, train_func, default_config

neptune_logger_tuner = NeptuneLogger(project=PROJECT_NAME, api_key=API_TOKEN, log_model_checkpoints=False, tags=["tune"])
tune_ckpt_path =  "/teamspace/studios/this_studio/.neptune/AUD1-389/AUD1-389/checkpoints/epoch=3946-step=27629.ckpt"

config = default_config
config["decoder"] = "greedy"

trainer = train_func(config=config, logger=neptune_logger_tuner, checkpoints=tune_ckpt_path, num_epochs=10000)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning_sdk/helpers.py:44: A newer version of lightning-sdk is available (0.1.15). Please consider upgrading with `pip install -U lightning-sdk`. Not all functionalities of the platform can be guaranteed to work with the current version.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/nn/modules/rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[neptune] [warning] NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also not

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/mrobay/Audio-project/e/AUD1-390


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at /teamspace/studios/this_studio/.neptune/AUD1-389/AUD1-389/checkpoints/epoch=3946-step=27629.ckpt
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:122: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:122: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 4 operations to synchronize with Neptune. Do not kill this process.


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


[neptune] [info   ] All 4 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/mrobay/Audio-project/e/AUD1-390/metadata


---
# Validation

---
# Test the classifier

In [ ]:
import importlib
importlib.reload(importlib.import_module("hebrewASR"))
importlib.reload(importlib.import_module("dataModule"))
from hebrewASR import HebrewASR, train_func, default_config
from hebrewASR import test_func, default_config

neptune_logger_test = NeptuneLogger(project=PROJECT_NAME, api_key=API_TOKEN, log_model_checkpoints=False, tags=["test"])

test_checkpoint = "/teamspace/studios/this_studio/.neptune/AUD1-363/AUD1-363/checkpoints/epoch=297-step=26522.ckpt"

config = default_config
config["decoder"] = "beam"

test_func(config=config, logger=neptune_logger_test, checkpoints=test_checkpoint)